## 12-7. Project: 멋진 챗봇 만들기
**라이브러리 버전을 확인해 봅니다**

---
사용할 라이브러리 버전을 둘러봅시다.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import re
import os
import io
import time
import random

import gensim
from collections import Counter
from konlpy.tag import Mecab

from sklearn.model_selection import train_test_split

지난 노드에서 **챗봇과 번역기는 같은 집안**이라고 했던 말을 기억하시나요?
앞서 배운 Seq2seq번역기와 Transfomer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해 봅시다. 배운 지식을 다양하게 활용할 수 있는 것도 중요한 능력이겠죠. 이번 프로젝트를 통해서 챗봇과 번역기가 같은 집안인지 확인해 보세요!

Step 1. 데이터 다운로드

---
준비하기 단계에서 심볼릭 링크를 생성했다면 아래 파일이 ```ChatbotData .csv```라는 이름으로 저장되어 있을거예요. ```csv``` 파일을 읽는 데에는 ```pandas``` 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 ```questions```, ```answers``` 변수에 나눠서 저장하세요!

- [songys/Chatbot_data](https://github.com/songys/Chatbot_data)


In [ ]:
data = pd.read_csv('~/aiffel/transformer_chatbot/data/ChatbotData .csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
questions = data['Q']
answers = data['A']

In [ ]:
print(len(questions), len(answers))
print('--------------------------')
print(questions.head())
print('--------------------------')
print(answers.head())

11823 11823
--------------------------
0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object
--------------------------
0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


Step 2. 데이터 정제

---
아래 조건을 만족하는 ```preprocess_sentence()``` 함수를 구현하세요.

1. 영문자의 경우, **모두 소문자로 변환**합니다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 **정규식을 활용하여 모두 제거**합니다.

*문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!*

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence)
    return sentence

Step 3. 데이터 토큰화

---
토큰화에는 *KoNLPy*의 ```mecab``` 클래스를 사용합니다.

아래 조건을 만족하는 ```build_corpus()``` 함수를 구현하세요!

1. **소스 문장 데이터**와 **타겟 문장 데이터**를 입력으로 받습니다.
2. 데이터를 앞서 정의한 **```preprocess_sentence()```** 함수로 **정제하고, 토큰화**합니다.
3. 토큰화는 **전달받은 토크나이즈 함수를 사용**합니다. 이번엔 ```**mecab.morphs**``` 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 **데이터에서 제외**합니다.
5. **중복되는 문장은 데이터에서 제외**합니다. ```소스 : 타겟``` 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!

구현한 함수를 활용하여 ```questions``` 와 ```answers``` 를 각각 ```que_corpus``` , ```ans_corpus``` 에 토큰화하여 저장합니다.

In [ ]:
mecab = Mecab()

In [ ]:
def build_corpus(src_data, tgt_data):
    mecab_src_corpus, mecab_tgt_corpus = [], []
    mecab_src_len_list, mecab_tgt_len_list = [], []
    
    for s, t in zip(src_data, tgt_data):
        s = mecab.morphs(preprocess_sentence(s))
        t = mecab.morphs(preprocess_sentence(t))
        
        mecab_src_corpus.append(s)
        mecab_tgt_corpus.append(t)
        
        mecab_src_len_list.append(len(s))
        mecab_tgt_len_list.append(len(t))

    mecab_num_tokens = mecab_src_len_list + mecab_tgt_len_list
    
    mean_len = np.mean(mecab_num_tokens)
    max_len = np.max(mecab_num_tokens)
    mid_len = np.median([mean_len, max_len])
    print(f'mid_len : {mid_len}')
    
    src_corpus, tgt_corpus = [], []
    for q, a in zip(mecab_src_corpus, mecab_tgt_corpus):
        if len(q) <= mid_len and len(a) <= mid_len:
            if q not in src_corpus and a not in tgt_corpus:
                src_corpus.append(q)
                tgt_corpus.append(a)
    
    return src_corpus, tgt_corpus

In [ ]:
que_corpus, ans_corpus = build_corpus(questions, answers)

mid_len : 23.849149961938593


In [ ]:
que_corpus[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네'],
 ['sd', '카드', '망가졌', '어']]

In [ ]:
ans_corpus[:5]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']]

In [ ]:
print(len(que_corpus))
print(len(ans_corpus))

7637
7637


Step 4. Augmentation

---
우리에게 주어진 데이터는 **1만 개가량으로 적은 편**에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? **Lexical Substitution을 실제로 적용**해 보도록 하겠습니다.

아래 링크를 참고하여 **한국어로 사전 훈련된 Embedding 모델을 다운로드**합니다. ```Korean (w)``` 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 ```Korean (w)```를 찾아 다운로드하고, ```ko.bin``` 파일을 얻으세요!

- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 **데이터를 Augmentation** 하세요! 앞서 정의한 ```lexical_sub()``` 함수를 참고하면 도움이 많이 될 겁니다.

*Augmentation*된 ```*que_corpus*``` 와 원본 ```*ans_corpus*``` 가 병렬을 이루도록, 이후엔 반대로 원본 ```*que_corpus*``` 와 Augmentation된 ```*ans_corpus*``` 가 병렬을 이루도록 하여 **전체 데이터가 원래의 3배가량으로 늘어나도록** 합니다.

In [ ]:
!pip install --upgrade gensim==3.8.3

In [ ]:
word2vec_path = os.getenv('HOME') + '/aiffel/transformer_chatbot/data/ko.bin'
word2vec = gensim.models.Word2Vec.load(word2vec_path)

In [ ]:
wv = gensim.models.Word2Vec.load('./data/ko.bin')

In [ ]:
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [ ]:
lexical_sub(que_corpus[0], wv)

/tmp/ipykernel_398/663423007.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]


'12 시 땡 캐치 '

In [ ]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], wv)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
    else:continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], wv)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
        
    else:continue

/tmp/ipykernel_398/645759352.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(que_corpus))):


  0%|          | 0/7637 [00:00<?, ?it/s]

/tmp/ipykernel_398/663423007.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]
/tmp/ipykernel_398/645759352.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(ans_corpus))):


  0%|          | 0/7637 [00:00<?, ?it/s]

In [ ]:
print(len(new_que_corpus))
print(len(new_ans_corpus))

13279
13279


In [ ]:
new_que_corpus[:5]

[['12', '시', '땡', '캐치'],
 ['3', '박', '4', '일', '놀', '러', '놀드', '고', '싶', '다'],
 ['ppl', '심하', '카나'],
 ['sns', '맞', '나르', '왜', '안', '하', '지'],
 ['sns', '시간', '낭비', '인', '거', '아', '는데', '매일', '시키', '는', '중']]

In [ ]:
new_ans_corpus[:5]

[['하루', '가', '또', '가', '네요', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.'],
 ['시간', '을', '정하', '고', '해', '보', '세요', '.']]

In [ ]:
from tqdm import tqdm_notebook

new_que_new_corpus = []
new_ans_new_corpus = []
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], wv)
    
    if ans_augmented is not None:
        new_que_new_corpus.append(que)
        new_ans_new_corpus.append(ans_augmented.split())
        
    else:continue
        
        
for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], wv)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_new_corpus.append(que_augmented.split())
        new_ans_new_corpus.append(ans)
    else:continue

/tmp/ipykernel_398/4156428745.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(ans_corpus))):


  0%|          | 0/7637 [00:00<?, ?it/s]

/tmp/ipykernel_398/663423007.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]
/tmp/ipykernel_398/4156428745.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(que_corpus))):


  0%|          | 0/7637 [00:00<?, ?it/s]

In [ ]:
print(len(new_que_new_corpus))
print(len(new_ans_new_corpus))

13230
13230


In [ ]:
new_que_new_corpus[:5]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['sd', '카드', '망가졌', '어'],
 ['sns', '맞', '팔', '왜', '안', '하', '지']]

In [ ]:
new_ans_new_corpus[:5]

[['하루', '가', '또', '가', '네요', '는데'],
 ['무릎', '해', '드립니다', '.'],
 ['항해', '은', '언제나', '좋', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '는데'],
 ['잘', '몰르', '고', '있', '을', '수', '도', '있', '어요', '.']]

Step 5. 데이터 벡터화

---
타겟 데이터인 ```ans_corpus``` 에 ```<start>``` 토큰과 ```<end>``` 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 ```ans_corpus``` 는 ```list``` 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!

In [ ]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

['<start>', '12', '시', '땡', '!', '<end>']


1. 위 소스를 참고하여 타겟 데이터 전체에 ```<start>``` 토큰과 ```<end>``` 토큰을 추가해 주세요!
챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 **소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것**이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

2. 특수 토큰을 더함으로써 ```ans_corpus``` 또한 완성이 되었으니, ```que_corpus``` 와 결합하여 **전체 데이터에 대한 단어 사전을 구축**하고 **벡터화하여 ```enc_train``` 과 ```dec_train```** 을 얻으세요!

In [ ]:
temp = []
    
for corpus in new_ans_new_corpus:
    temp.append(["<start>"] + corpus + ["<end>"])

In [ ]:
ans_corpus = temp

In [ ]:
len(ans_corpus)

13230

In [ ]:
ans_corpus[:5]

[['<start>', '하루', '가', '또', '가', '네요', '는데', '<end>'],
 ['<start>', '무릎', '해', '드립니다', '.', '<end>'],
 ['<start>', '항해', '은', '언제나', '좋', '죠', '.', '<end>'],
 ['<start>', '다시', '새로', '사', '는', '게', '마음', '편해요', '는데', '<end>'],
 ['<start>', '잘', '몰르', '고', '있', '을', '수', '도', '있', '어요', '.', '<end>']]

In [ ]:
total_data = new_que_new_corpus + ans_corpus
len(total_data)

26460

In [ ]:
words = np.concatenate(total_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

In [ ]:
word_to_index

{'<pad>': 0,
 '<unk>': 1,
 '.': 2,
 '<start>': 3,
 '<end>': 4,
 '이': 5,
 '는': 6,
 '하': 7,
 '을': 8,
 '가': 9,
 '좋': 10,
 '고': 11,
 '세요': 12,
 '어': 13,
 '있': 14,
 '거': 15,
 '은': 16,
 '해': 17,
 '는데': 18,
 '보': 19,
 '지': 20,
 '?': 21,
 '아': 22,
 '나': 23,
 '도': 24,
 '게': 25,
 '겠': 26,
 '사람': 27,
 '에': 28,
 '를': 29,
 '예요': 30,
 '사랑': 31,
 '어요': 32,
 '같': 33,
 '죠': 34,
 '한': 35,
 '것': 36,
 '없': 37,
 '싶': 38,
 '면': 39,
 '수': 40,
 '네요': 41,
 '다': 42,
 '의': 43,
 '네': 44,
 '안': 45,
 '친구': 46,
 '생각': 47,
 '않': 48,
 '봐요': 49,
 '아요': 50,
 '마음': 51,
 '말': 52,
 '할': 53,
 '되': 54,
 '주': 55,
 '잘': 56,
 '너무': 57,
 '남자': 58,
 '했': 59,
 '이별': 60,
 '더': 61,
 '기': 62,
 '었': 63,
 '내': 64,
 '연락': 65,
 '만': 66,
 '여자': 67,
 '일': 68,
 '들': 69,
 '힘들': 70,
 '남': 71,
 '해요': 72,
 '썸': 73,
 '그러': 74,
 '많이': 75,
 '시간': 76,
 '짝': 77,
 '으로': 78,
 '괜찮': 79,
 '한테': 80,
 '길': 81,
 '았': 82,
 '에서': 83,
 '으면': 84,
 '에요': 85,
 '건': 86,
 '때': 87,
 '에게': 88,
 '야': 89,
 '그': 90,
 '만나': 91,
 '알': 92,
 '시키': 93,
 '던': 94,
 '은데': 95,


In [ ]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

In [ ]:
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])

In [ ]:
def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

In [ ]:
que_train = vectorize(new_que_new_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

In [ ]:
enc_train = keras.preprocessing.sequence.pad_sequences(que_train, padding='pre', maxlen=20)
dec_train = keras.preprocessing.sequence.pad_sequences(ans_train, padding='pre', maxlen=20)

In [ ]:
enc_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 3208,  230, 3728,  116], dtype=int32)

In [ ]:
dec_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
       288,   9, 149,   9,  41,  18,   4], dtype=int32)

 1. 위 소스를 참고하여 타겟 데이터 전체에 '**\<start**\>' 토큰과 '**\<end\>**' 토큰을 추가

- 챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것
- 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있음

  - 특수 토큰을 더함으로써 'ans_corpus' 또한 완성이 되었으니, 'que_corpus'와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 'enc_train'과 'dec_train'을 얻기

Step 6. 훈련하기

---
앞서 번역 모델을 훈련하며 정의한 ```Transformer``` 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! **가장 멋진 답변**과 **모델의 하이퍼파라미터**를 제출하시면 됩니다.



In [ ]:
#Positional Encoding
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [ ]:
#마스크 생성
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [ ]:
#Multi-head Attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [ ]:
#Position-wise Feed Forward Network
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

In [ ]:
#Encoder Layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

In [ ]:
#Decoder Layer
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [ ]:
#Encoder
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [ ]:
# Decoder
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [ ]:
#Transformer 전체 모델 조립
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [ ]:
# 모델 인스턴스 생성
VOCAB_SIZE = 30000

transformer = Transformer(
    n_layers=5,
    d_model=512,
    n_heads=16,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [ ]:
#Learning Rate Scheduler
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [ ]:
#Learing Rate & Optimizer
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [ ]:
#Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
#Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]
    gold = tgt[:, 1:]

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [ ]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 128
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/tmp/ipykernel_398/1860345807.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  t = tqdm_notebook(idx_list)


  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

...

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]


```
# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

# 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
```

In [ ]:
def evaluate(sentence, model):
    mecab = Mecab()
    
    sentence = preprocess_sentence(sentence)
    pieces = mecab.morphs(sentence)
    
    tokens = []
    for sen in pieces:
        sen= get_encoded_sentence(sen, word_to_index)
        tokens.append(sen)
    
    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                        value=word_to_index["<pad>"],
                                                        padding='pre',
                                                        maxlen=20)
    
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)

    return result

In [ ]:
samples = ["지루하다, 놀러가고 싶어.",
           "오늘 일찍 일어났더니 피곤하다.",
           "간만에 여자친구랑 데이트 하기로 했어.",
           "집에 있는다는 소리야."]

In [ ]:
predicts=[]
for sample in samples:
    predicts.append(translate(sample, transformer))
    print('sample : ', sample)
    print('Translations : ', translate(sample, transformer))

sample :  지루하다, 놀러가고 싶어.
Translations :  하 는 건 위로 하 는 건 위로 하 는 건 위로 하 는 건 위로 하
sample :  오늘 일찍 일어났더니 피곤하다.
Translations :  하 지 마요 . 하 지 마요 . 하 게 좋 은 모두 하 세요 . 마요 .
sample :  간만에 여자친구랑 데이트 하기로 했어.
Translations :  중 했 다면 정말 하 는 사람 이 있 다면 정말 정말 정말 정말 정말 하 니까요 .
sample :  집에 있는다는 소리야.
Translations :  은 힘내 세요 . 하 고 하 고 좋 은 힘내 하 고 좋 겠 네요 .


In [ ]:
translations=["잠깐 쉬 어도 돼요 . <end>", "맛난 거 드세요 . <end>", "떨리 겠 죠 . <end>", "좋 아 하 면 그럴 수 있 어요 . <end>"]

(1) BLEU Score

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [ ]:
def calculate_bleu(translations, predicts, weights=[0.25, 0.25, 0.25, 0.25]):
    for t, p in zip(translations, predicts):
        print("BLEU Score: ", sentence_bleu([t.split()],
                                            p.split(),
                                            weights=weights,
                                            smoothing_function=SmoothingFunction().method1))
        
calculate_bleu(translations, predicts)

BLEU Score:  0
BLEU Score:  0.010802314890908065
BLEU Score:  0.010802314890908065
BLEU Score:  0.015138514598766055


## 루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

|평가문항|상세기준|
|:-----|:------|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|

# 회고

이전의 익플 6에서 영화감성분석 리뷰가 떠오른던
```ko.bin```을 다시 만나게 되었다.

이게 파일 불러오기가 안되던 중 혜령님이 예전 노드에 겐심을 다운그레이드해야한다고 알려주셨다.!(바로 해결!)


솔직히 nlp가 너무 어려워서 그 전 기수분들 노드를 참고안하면 진행하기가 너무 힘들었다. 

카피카피닌자의 능력을 부여받아 어찌저찌했지만👻<br>
제대로 된 문장을 만들어내는것은 그 이후의 일.. 너무 어렵다😥

패딩은 pre랑 post 둘 중 post가 좀 더 잘 나왔던 것 같다.
큰 문제점은 하 다 를 를 같은 형태소가 단어와 단어사이를 메꿔야하는데 줄줄이 뒤를 이어나오는게.. 이건 데이터 정제할 때 고려를 했어야하는 사항이었던 것 같다.

nlp는 언제나 이렇듯 완벽한 문장하나 내는게 이렇게 힘든 것인걸

BLEU Score를 구하는건 하라고 해서 구하기는 했지만 제대로 된 사용을 모르겠다.<br>
번역기를 만들었다면 제대로 나왔는지 비교를 할 수는 있지만 챗봇에서의 사용은 정해진 답이 있는 것도 아니라서 BLEU를 구하는게 맞는지 의문이 들었다.

🌈🌈🌈그리고 갖은 에러와 난관에 부딪쳤을 때, 어김없이 도와준 혜령님께 슴슴한 감사를 드립니다
